In [ ]:
import urllib.request, urllib.parse, urllib.error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import locale
import os
import matplotlib.patches as mpatches
import sys
from numpy.random import seed
import datetime

pd.options.display.max_columns=50
seed(888)

today = datetime.date.today() + datetime.timedelta(days=1) # +1 day
dataT = str(today)

pathP='/mnt/Production/СМП'

In [ ]:
# dataT

In [ ]:
try:
    ttt = int(sys.argv[1:][0])
except:
    ttt = 10
    
print('args', ttt)


if ttt==0: teritors=['Ленинградская область', 1]
if ttt==1: teritors=['Волгоградская область', 1]
if ttt==2: teritors=['Мурманская область', 1]
if ttt==3: teritors=['Тульская область', 1]
if ttt==4: teritors=['Ставропольский край', 1]
if ttt==5: teritors=['Пермский край', 1]
if ttt==6: teritors=['Краснодарский край', 1]
if ttt==7: teritors=['Свердловская область', 1]


if ttt==8: teritors=['Кемеровская область', 2]
if ttt==9: teritors=['Алтайский край', 2]
if ttt==10: teritors=['Красноярский край', 2]
if ttt==11: teritors=['Республика Хакасия', 2]
if ttt==12: teritors=['Новосибирская область', 2]
if ttt==13: teritors=['Забайкальский край', 2]
if ttt==14: teritors=['Республика Тыва', 2]
if ttt==15: teritors=['Республика Бурятия', 2]
    
if ttt==16: teritors=['Хабаровский край', 0]
if ttt==17: teritors=['Приморский край', 0]

    
print('-------------',teritors,'-------------')

In [ ]:
ter = teritors[0]

In [ ]:
# ter

In [ ]:
import pyodbc
#DB Credentianals
server = r'10.79.226.101'
database = r'TradeInfo'
username = r'sibgenco.local\omima'
password = r'VxjNOJV]!3'
conn = pyodbc.connect('DRIVER={FreeTDS};SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password+';TDS_VERSION=8.0;')

In [ ]:
cursor = conn.cursor()

query = "SELECT TOP 1 * FROM SrcSCH WHERE FC_DAY = '" + dataT + "' AND REGION = '" + ter + "' AND ModelName = 'v1' ORDER BY SENT_TIME DESC"
data = pd.read_sql(query, conn)
cursor.close()
# data

In [ ]:
# последнии данные
d_now = [data['HOUR1'].values[0], data['HOUR2'].values[0], data['HOUR3'].values[0]]
print(d_now)

In [ ]:
#ID записи часов прогноза
id_now = data['Id'].values[0]
print('id', id_now)

In [ ]:
# str(datetime.datetime.now())

In [ ]:
# Задаем ценовую зону регионам. 1 - первая ценовая, 2 - вторая ценовая, 0 - 2 неценовая зона
# Можно регулировать порядок вывода графиков меняя порядок
region_zones = {
    'Республика Хакасия' : 2,
    'Алтайский край' : 2,
    'Красноярский край' : 2,
    'Кемеровская область' : 2,    
    'Забайкальский край' : 2,
    'Новосибирская область' : 2,
    'Республика Тыва' : 2,
    'Республика Бурятия' : 2,
    'Волгоградская область' : 1,
    'Краснодарский край' : 1,    
    'Ленинградская область' : 1,
    'Мурманская область' : 1,    
    'Пермский край' : 1,
    'Свердловская область' : 1,
    'Ставропольский край' : 1,
    'Тульская область' : 1,
    'Хабаровский край' : 0,
    'Приморский край' : 0,
   }

In [ ]:
# Плановые часы пиковой нагрузки на 2021 год для территорий, отнесенных к ценовым зонам оптового рынка
# электрической энергии и мощности, и территорий, отнесенных к неценовым зонам оптового рынка
# электрической энергии и мощности
#
# Здесь задаются часы для первой и второй ценовых зон
# Значение 0.6 означает час пиковой нагрузки, порядок идет с 1 по 24 час
# Список часов задается для каждого месяца с 1 по 12 (янв - дек)
# 
# Источник:
# https://www.so-ups.ru/fileadmin/files/company/markets/2021/pik_chas2021.pdf
# 
zones = {       
    1 : {
        1 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        2 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        3 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        4 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        5 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0,    0,    0],
        6 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.6,  0.6,  0,    0,    0],
        7 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.6,  0.6,  0,    0,    0],
        8 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        9 : [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        10: [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        11: [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
        12: [ 0,   0,   0,   0,   0,   0,   0,  0.6, 0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0,    0,    0],
    },      
    2 :{
        1 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        2 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        3 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        4 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        5 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        6 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.6,  0.6,  0.6,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        7 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        8 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.6,  0.6,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        9 : [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        10: [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        11: [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
        12: [ 0,   0,   0,   0, 0.6, 0.6, 0.6,  0.6, 0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0,    0,    0],
    },
    0 :{
        1 : [ 0,   0,   0,   0,   0,   0,    0,   0,   0,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.0,  0],
        2 : [ 0,   0,   0,   0,   0,    0,   0,   0,   0,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0],
        3 : [ 0,   0,   0,   0,   0,    0,   0,   0, 0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0],
        4 : [ 0,   0,   0,   0,   0,    0,   0,   0,   0,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0],
        5 : [ 0,   0,   0,   0,   0,    0,   0,   0,   0,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0],
        6 : [ 0,   0,   0,   0,   0,    0,   0,   0,   0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0],
        7 : [ 0,   0,   0,   0,   0,    0,   0,   0,   0,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0],
        8 : [ 0,   0,   0,   0,   0,    0,   0,   0,   0,    0,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.6,  0],
        9 : [ 0,   0,   0,   0,   0,    0,   0,   0,   0,    0,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0],
        10: [ 0,   0,   0,   0,   0,    0,   0,   0,   0,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0],
        11: [ 0,   0,   0,   0,   0,    0,   0,   0,   0,  0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.0,  0],
        12: [ 0,   0,   0,   0,   0,    0,   0,   0, 0.6,  0.6,  0.6,  0.6,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.6,  0.6,  0.6,  0.6,  0.0,  0],
    }
}

In [ ]:
# Текущий месяц
cur_month = today.month
# Название месяца
cur_month_name = today.strftime("%B")

hour_pref = ''
hour_name = 'время московское'
if ttt in [16, 17]:
    hour_pref = '+7'
    hour_name = 'время хабаровское'

# Построение графиков и сохранение в PDF
fig = plt.figure(figsize=(9.54, 42.39), dpi=200)
plt.suptitle("Контрольные часы на " + today.strftime("%d.%m.%Y") + " (МСК" + hour_pref + ")", va="bottom", y=0.885, fontsize=18, fontweight=600, horizontalalignment='center')

# red_patch = mpatches.Patch(color='#e7344c', label='Прогнозный час вне пикового')
# green_patch = mpatches.Patch(color='green', label='Пиковые часы')
navy_patch = mpatches.Patch(color='#5f8ab8', label='Первый прогнозный час')
# blue1_patch = mpatches.Patch(color='#061D95', label='Прогнозный час 2')
# blue2_patch = mpatches.Patch(color='#0D3AA9', label='Прогнозный час 3')
pik_patch = mpatches.Patch(color='#FED966', label='Пиковые часы')
pr_patch = mpatches.Patch(color='#FF7C81', label='Прогнозный час')

# print()
print(f"Текущая дата: {today.day} {cur_month_name} {today.year}")
# print()
counter = 0
indexer = 0
# region_zones

for reg in [ter]:
#     print(reg)
    
    num_zone = region_zones[reg]
    data = zones[num_zone][cur_month]
    actual_data = []
    colors = []
    
    for i in range(1, 24 + 1):
        if i == d_now[0]:
            actual_data.append(0.72)
            #print(data[i - 1])
            if data[i - 1] == 0.6:
                colors.append('#5f8ab8')
#                 colors.append('#FF7C81')
            else:
                colors.append('#e7344c')
        elif i == d_now[1]:
            actual_data.append(0.6)
            #print(data[i - 1])
            if data[i - 1] == 0.6:
#                 colors.append('#061D95')
                colors.append('#FF7C81')
            else:
                colors.append('#e7344c')
        elif i == d_now[2]:
            actual_data.append(0.6)
            #print(data[i - 1])
            if data[i - 1] == 0.6:
#                 colors.append('#0D3AA9')
                colors.append('#FF7C81')
            else:
                colors.append('#e7344c')
        else:
            #print(data[i - 1])
            actual_data.append(data[i - 1])
            #colors.append('#ffd430')
            colors.append('#FED966')

    # Создание пути для сохранения файла
    path = './pdfs/'
    if not os.path.exists(path + today.strftime("%Y")):        
        os.mkdir(path + today.strftime("%Y"))
    if not os.path.exists(path + today.strftime("%Y") + '/' + today.strftime("%m")):        
        os.mkdir(path + today.strftime("%Y") + '/' + today.strftime("%m"))
    if not os.path.exists(path + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d")):        
        os.mkdir(path + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d"))
#     if counter%3 == 0 and counter > 0:
#         print('!')
#         indexer = 3
#     else:
#         indexer = 0
    counter += 1
#     print(11, 3, counter + indexer)
    plt.subplot(16, 1, counter + indexer)
    
    
    # Столбцы часов
    x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
    y = actual_data
    
    actual_data[0] = 1
    
    colors[0] = 'white'
    edgecolors = ['white','black','black','black','black','black','black','black','black','black',
                 'black','black','black','black','black','black','black','black','black','black','black','black','black','black']
    
    plt.bar(x, y, width=1, align='center', color=colors, edgecolor=edgecolors )
    plt.text(12, 0.8, f"{reg}", 
              fontdict={'fontsize': 16,
                        'fontweight': 600,
                        'color': 'black',
                        'verticalalignment': 'baseline',
                        'horizontalalignment': 'center'}
             )  
    plt.yticks(y, " ")

    #     plt.show()
    #     print()
#     if counter%3 == 0 and counter > 0:
#         indexer += 1
#         print(indexer)
#         print(11, 3, counter )
#         plt.subplot(11, 1, counter+1)
#         plt.title(indexer)

# plt.subplot(16, 1, 16)
# plt.figure(figsize=(9.54, 2), dpi=200)
plt.legend(handles=[navy_patch, pik_patch, pr_patch], ncol=3, loc='upper center', bbox_to_anchor=(0.5, -0.25))
# plt.axis("off")
# print(fig.get_size_inches()  )
# Сохраняем графики
plt.savefig(path + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + 'Контрольные часы ' + ter + ' на ' + today.strftime("%d-%m-%Y") + '.pdf', dpi=200, bbox_inches='tight', pad_inches=0.2)


In [ ]:
import smtplib                                              # Импортируем библиотеку по работе с SMTP
import ssl                                                  # Библиотека для работы с SSL
import os                                                   # Функции для работы с операционной системой, не зависящие от используемой операционной системы
 
# Добавляем необходимые подклассы - MIME-типы
import mimetypes                                            # Импорт класса для обработки неизвестных MIME-типов, базирующихся на расширении файла
from email import encoders                                  # Импортируем энкодер
from email.mime.base import MIMEBase                        # Общий тип
from email.mime.text import MIMEText                        # Текст/HTML
from email.mime.image import MIMEImage                      # Изображения
from email.mime.audio import MIMEAudio                      # Аудио
from email.mime.multipart import MIMEMultipart              # Многокомпонентный объект
 

def send_email(addr_to, msg_subj, msg_text, files):
    addr_from = "omima@suek.ru"                             # Отправитель
    password  = r'VxjNOJV]!3'                               # Пароль
 
    msg = MIMEMultipart()                                   # Создаем сообщение
    msg['From']    = addr_from                              # Адресат
    msg['To']      = addr_to                                # Получатель
    msg['Subject'] = msg_subj                               # Тема сообщения
 
    body = msg_text                                         # Текст сообщения
    msg.attach(MIMEText(body, 'plain'))                     # Добавляем в сообщение текст
 
    process_attachement(msg, files)
 
    #==========================================================================================================================
    # Create a secure SSL context
    context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
    # Try to log in to server and send email
    try:
        server = smtplib.SMTP("kmr-relay.suek.ru", 587)      # Создаем объект SMTP
        server.ehlo()                                           # Can be omitted
        server.starttls(context=context)                        # Начинаем шифрованный обмен по TLS
#         server.set_debuglevel(True)                            # Включаем режим отладки, если не нужен - можно закомментировать
        server.ehlo()                                           # Can be omitted
        server.login(addr_from, password)                       # Получаем доступ
        server.send_message(msg)                                # Отправляем сообщение
        server.quit()                                           # Выходим
    except Exception as e:
        # Print any error messages to stdout
        print('Возникла проблема при отправке. Ошибка: ' + str(e))        
    #==========================================================================================================================

def process_attachement(msg, files):                        # Функция по обработке списка, добавляемых к сообщению файлов
    for f in files:
        if os.path.isfile(f):                               # Если файл существует
            attach_file(msg,f)                              # Добавляем файл к сообщению
        elif os.path.exists(f):                             # Если путь не файл и существует, значит - папка
            dir = os.listdir(f)                             # Получаем список файлов в папке
            for file in dir:                                # Перебираем все файлы и...
                attach_file(msg,f+"/"+file)                 # ...добавляем каждый файл к сообщению

def attach_file(msg, filepath):                             # Функция по добавлению конкретного файла к сообщению
    filename = os.path.basename(filepath)                   # Получаем только имя файла
    ctype, encoding = mimetypes.guess_type(filepath)        # Определяем тип файла на основе его расширения
    if ctype is None or encoding is not None:               # Если тип файла не определяется
        ctype = 'application/octet-stream'                  # Будем использовать общий тип
    maintype, subtype = ctype.split('/', 1)                 # Получаем тип и подтип
    if maintype == 'text':                                  # Если текстовый файл
        with open(filepath) as fp:                          # Открываем файл для чтения
            file = MIMEText(fp.read(), _subtype=subtype)    # Используем тип MIMEText
            fp.close()                                      # После использования файл обязательно нужно закрыть
    elif maintype == 'image':                               # Если изображение
        with open(filepath, 'rb') as fp:
            file = MIMEImage(fp.read(), _subtype=subtype)
            fp.close()
    elif maintype == 'audio':                               # Если аудио
        with open(filepath, 'rb') as fp:
            file = MIMEAudio(fp.read(), _subtype=subtype)
            fp.close()
    else:                                                   # Неизвестный тип файла
        with open(filepath, 'rb') as fp:
            file = MIMEBase(maintype, subtype)              # Используем общий MIME-тип
            file.set_payload(fp.read())                     # Добавляем содержимое общего типа (полезную нагрузку)
            fp.close()
            encoders.encode_base64(file)                    # Содержимое должно кодироваться как Base64
    file.add_header('Content-Disposition', 'attachment', filename=filename) # Добавляем заголовки
    msg.attach(file)                                        # Присоединяем файл к сообщению


files = [path + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + 'Контрольные часы ' + ter + ' на ' + today.strftime("%d-%m-%Y") + '.pdf', # Список файлов, если вложений нет, то files=[]
        # "file2_path",                                      
        # "dir_path"  # Если нужно отправить все файлы из заданной папки, нужно указать её
        ]       

# text_mail = ter + "\n\nКонтрольные часы на " + today.strftime("%d.%m.%Y") + " (МСК" + hour_pref + ")\n\n" + "Это сообщение сформировано автоматически, отвечать на него не нужно."

text_mail = "Прогноз контрольных часов (часы максимума, СКЧ) на " + today.strftime("%d.%m.%Y") + " (МСК" + hour_pref + ") по региону: " + ter + ".\n\n"
text_mail += "При планировании графика работы оборудования на " + today.strftime("%d.%m.%Y") + " обратите внимание, что прогнозные часы по региону: " + str(d_now[0]) + ", " + str(d_now[1]) + ", " + str(d_now[2]) + " (" + hour_name + ").\n"
text_mail += "В указанные часы рекомендуется принимать меры по снижению потребления электроэнергии.\n\n"
text_mail += "Это сообщение сформировано автоматически, отвечать на него не нужно.\n\n"
text_mail += "При необходимости корректировки списка рассылки обращаться к Богданову А.В. по Email: BogdanovAndV@sibgenco.ru .\n\n"
text_mail += "Это сообщение и все приложения к нему строго конфиденциальны и предназначены исключительно для использования получателем(адресатом), которому они адресованы. Использование информации, содержащейся в настоящем сообщении, допускается с соблюдением требований законодательства РФ об охране конфиденциальной информации и соглашений о коммерческой тайне."


In [ ]:
addr_1cz = pd.read_excel(pathP + '/СКЧ списки на отправку.xlsx', sheet_name='1цз')
# addr_1cz

In [ ]:
addr_2cz = pd.read_excel(pathP + '/СКЧ списки на отправку.xlsx', sheet_name='2цз')
# addr_2cz

In [ ]:
addr_Ncz = pd.read_excel(pathP + '/СКЧ списки на отправку.xlsx', sheet_name='нцз')
# addr_Ncz

In [ ]:
addr_cz = pd.concat([addr_1cz, addr_2cz, addr_Ncz], axis=1)
# addr_cz

In [ ]:
addr_dict = {}

for reg in region_zones:
    addr_dict[reg] = []
    for value in addr_cz[reg].dropna().values:
#         print(value.split()[-1:][0])
        addr_dict[reg].append(value.split()[-1:][0])
    addr_dict[reg] = ",".join(addr_dict[reg])
# addr_dict

In [ ]:
# Получатель
addr_to   = addr_dict[ter]
# addr_to   = 'VelichkoYA@sibgenco.ru'
# addr_to   = 'BogdanovAndV@sibgenco.ru'  

In [ ]:
print('emails =',addr_to)

In [ ]:
# Использование функции send_email()
status = 'send'
if status == 'send':
    send_email(addr_to, "Контрольные часы " + ter + " на " + today.strftime("%d.%m.%Y") + " (МСК" + hour_pref + ")", text_mail, files)
    print('sended')

In [ ]:
#доп отправка xml-файла

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def xml_creator(predict_hours, SO_hours, day, ter):
    hours = pd.DataFrame(SO_hours[day.month])
    hours = hours.loc[(hours[0] > 0)]
    look = []
    look = hours.index

    minh_1 = min(look)
    maxh_2 = max(look) + 1
    maxh_1 = 0
    for i in range(1, len(look)):
        if look[i] - look[i - 1] > 1:
            maxh_1 = look[i - 1] + 1

    look_new = [hour for hour in look if hour > maxh_1]     

    minh_2 = min(look_new) # + 1

#     print(minh_1, maxh_1, minh_2, maxh_2)

    data = ET.Element('data')
    data.set('timezone','GMT+3')
    data.set('date', day.strftime('%d.%m.%Y'))
    data.set('region', ter)

    planned_peak_hours = ET.SubElement(data, 'planned-peak-hours')


    for i in range(1, 2 + 1):
        if i == 1:
            range_p_h_1 = ET.SubElement(planned_peak_hours, 'range')
            range_p_h_1.set('to', f"{maxh_1:02d}:00")
            range_p_h_1.set('from', f'{minh_1:02d}:00')
        else:
            range_p_h_1 = ET.SubElement(planned_peak_hours, 'range')
            range_p_h_1.set('to', f'{maxh_2:02d}:00')
            range_p_h_1.set('from', f'{minh_2:02d}:00')
            

    region_forecast_max_hours = ET.SubElement(data, 'region-forecast-max-hours')

    for i in range (1, len(predict_hours) + 1):
            hour_rfmh = ET.SubElement(region_forecast_max_hours, 'hour')
            hour_rfmh.set('to', f'{predict_hours[i - 1]:02d}:00')
            hour_rfmh.set('from', f'{predict_hours[i - 1] - 1:02d}:00')
            hour_rfmh.set('power-order', f'{i}')

    return data

In [ ]:
zn = zones[2]
#получатели и зона

if ttt == 10:
    addr_xml_to = 'fcthours_krsk@suek.ru,bogdanovandv@sibgenco.ru,velichkoya@sibgenco.ru'
    zn = zones[2]
if ttt == 8:
    addr_xml_to = 'vodootliv_koms@suek.ru,VodootlivHMM@suek.ru,bogdanovandv@sibgenco.ru,velichkoya@sibgenco.ru'
    zn = zones[2]
if ttt == 15:
    addr_xml_to = 'aiiskue-tur@suek.ru,bogdanovandv@sibgenco.ru,velichkoya@sibgenco.ru'
    zn = zones[2]
# addr_xml_to = 'bogdanovandv@sibgenco.ru'

# 10 - Красноярский край, 8 - Кемеровская, 15 - Бурятия
if ttt in [8, 10, 15]:    
    data = xml_creator(d_now, zn, today, ter)    
    xml_str = ET.tostring(data, encoding="utf-8", method="xml")
    print(xml_str.decode(encoding="utf-8"))

    # Создание пути для сохранения файла xml
    pathX = './xmls/'
    if not os.path.exists(pathX + today.strftime("%Y")):        
        os.mkdir(pathX + today.strftime("%Y"))
    if not os.path.exists(pathX + today.strftime("%Y") + '/' + today.strftime("%m")):        
        os.mkdir(pathX + today.strftime("%Y") + '/' + today.strftime("%m"))
    if not os.path.exists(pathX + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d")):        
        os.mkdir(pathX + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d"))
    if not os.path.exists(pathX + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + ter):
        os.mkdir(pathX + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + ter)
    
    #запись файла xml    
    ET.ElementTree(data).write(pathX + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + ter + '/peaks.xml', encoding='unicode', xml_declaration=True, method='xml')
        
    #выбор файла xml
    xml_file = [pathX + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + ter + '/peaks.xml']
    
    #отправка
    send_email(addr_xml_to, 
               "Контрольные часы " + ter + " на " + today.strftime("%d.%m.%Y") + " (МСК) - xml",
               "XML-файл во вложении.\n\nЭто сообщение сформировано автоматически, отвечать на него не нужно.\n\nЭто сообщение и все приложения к нему строго конфиденциальны и предназначены исключительно для использования получателем(адресатом), которому они адресованы. Использование информации, содержащейся в настоящем сообщении, допускается с соблюдением требований законодательства РФ об охране конфиденциальной информации и соглашений о коммерческой тайне.",
               xml_file)

In [ ]:
#обновлении информации об отправленных часах
cursor = conn.cursor()
query = "UPDATE SrcSCH SET GivenToUser = 1 WHERE Id = " + str(id_now)
print(query)
cursor.execute(query)
    
conn.commit()

In [ ]:
#запись в excel

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Fill

# pref = ''
# if teritors[1] == 0:
#     pref = '2N'
# else:
#     pref = teritors[1]

# prognoz = pd.DataFrame([d_now], columns=['V1', 'V2', 'V3'], index=[dataT])

# book = load_workbook(str(pathP + '/rez_' + str(pref) + 'cz.xlsx'))
# writer = pd.ExcelWriter(pathP + '/rez_' + str(pref) + 'cz.xlsx', engine='openpyxl')
# writer.book = book
# writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
# prognoz[['V1', 'V2', 'V3']].to_excel(writer, ter)

# sheet2 = book.get_sheet_by_name(ter)
# sheet2['A1'] = ter
# sheet2['A1'].font = Font(bold=True)

# sheet2['B2'].font = Font(color="FF0000", bold=True, size=14)
# sheet2['C2'].font = Font(color="FF0000", bold=True, size=14)
# sheet2['D2'].font = Font(color="FF0000", bold=True, size=14)
# for i in range(3, 7):
#     sheet2['B' + str(i)].font = Font(color="008000", bold=True, size=14)
#     sheet2['C' + str(i)].font = Font(color="008000", bold=True, size=14)
#     sheet2['D' + str(i)].font = Font(color="008000", bold=True, size=14)

# sheet2.column_dimensions["A"].width = 25.0

# #sheet2['B2'].font = Font(size=16)

# writer.save()

In [ ]:
pref = ''
if teritors[1] == 0:
    pref = '2N'
else:
    pref = teritors[1]

prognoz = pd.DataFrame([d_now], columns=['V1', 'V2', 'V3'], index=[dataT])

df = pd.read_excel(pathP + '/log/rez_' + str(pref) + 'cz.xlsx', sheet_name=ter, index_col=0)

prognoz = pd.concat([df, prognoz])

book = load_workbook(str(pathP + '/log/rez_' + str(pref) + 'cz.xlsx'))
writer = pd.ExcelWriter(pathP + '/log/rez_' + str(pref) + 'cz.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
prognoz[['V1', 'V2', 'V3']].to_excel(writer, ter)

# sheet2 = book.get_sheet_by_name(ter)
# sheet2['A1'] = ter
# sheet2['A1'].font = Font(bold=True)

# sheet2['B2'].font = Font(color="FF0000", bold=True, size=14)
# sheet2['C2'].font = Font(color="FF0000", bold=True, size=14)
# sheet2['D2'].font = Font(color="FF0000", bold=True, size=14)
# for i in range(3, 7):
#     sheet2['B' + str(i)].font = Font(color="008000", bold=True, size=14)
#     sheet2['C' + str(i)].font = Font(color="008000", bold=True, size=14)
#     sheet2['D' + str(i)].font = Font(color="008000", bold=True, size=14)

# sheet2.column_dimensions["A"].width = 25.0

# #sheet2['B2'].font = Font(size=16)

writer.save()